# Platypus2-70B + Wikipedia RAG


In [1]:
import gc
import logging
from time import time
from pathlib import Path
import ctypes
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm


# For LLM
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModel
from accelerate import init_empty_weights
from accelerate.utils.modeling import set_module_tensor_to_device
from safetensors.torch import load_file

N_BATCHES = 5
MAX_CONTEXT = 3000
MAX_LENGTH = 4096

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Function to clean RAM & vRAM
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()



## 2: Run Platypus2-70B

To run such a large model on a single T4 GPU, we run it layer by layer and sample by sample

In [3]:
# Create symlinks from kaggle datasets to fake cached model

checkpoint_path = Path("/root/.cache/")
checkpoint_path.mkdir(exist_ok=True, parents=True)

for part in [1, 2]:
    source_dir = Path(f"/kaggle/input/platypus2-70b-instruct-part{part}")
    for path in source_dir.glob("*"):
        try:
            (checkpoint_path / path.name).symlink_to(path)
        except:
            pass

In [4]:
# Class for sharded llama

class ShardedLlama:
    def __init__(self, checkpoint_path, device="cuda:0", dtype=torch.float16):
        """
        Sharded version of LlamaForCausalLM : the model is splitted into layer shards to reduce GPU memory usage.
        During the forward pass, the inputs are processed layer by layer, and the GPU memory is freed after each layer.
        To avoid loading the layers multiple times, we could save all the intermediate activations in RAM, but
        as Kaggle accelerators have more GPU memory than CPU, we simply batch the inputs and keep them on the GPU.

        Parameters
        ----------
        checkpoint_path : str or Path
            path to the checkpoint
        device : str, optional
            device, by default "cuda:0"
        dtype : torch.dtype, optional
            dtype, by default torch.float16
        """
        
        # Save parameters
        self.checkpoint_path = Path(checkpoint_path)
        self.device = device 
        self.dtype = dtype

        # Create model
        self.config = AutoConfig.from_pretrained(self.checkpoint_path)
        # For flash attention when Turing architecture will be supported : https://github.com/Dao-AILab/flash-attention/issues/542
        # self.config.auto_map = {"AutoModelForCausalLM" : "togethercomputer/LLaMA-2-7B-32K--modeling_flash_llama.LlamaForCausalLM"} 
        
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"
        self.init_model()
        self.layer_names = ["model.embed_tokens"] + [f"model.layers.{i}" for i in range(len(self.model.model.layers))] + ["model.norm", "lm_head"]

    def init_model(self):
    
        # Load meta model (no memory used)
        with init_empty_weights():
            self.model = AutoModelForCausalLM.from_config(self.config, trust_remote_code=True)
            self.model.tie_weights()
            
        self.layers = [self.model.model.embed_tokens] + list(self.model.model.layers) + [self.model.model.norm, self.model.lm_head]
            
        # Move buffers to device (not that much GPU memory used)
        for buffer_name, buffer in self.model.named_buffers():
            set_module_tensor_to_device(self.model, buffer_name, self.device, value=buffer, dtype=self.dtype)

    def load_layer(self, layer_name):
        state_dict = load_file(self.checkpoint_path / (layer_name + ".safetensors"), device=self.device)
        for param_name, param in state_dict.items():
            assert param.dtype != torch.int8, "int8 not supported (need to add fp16_statistics)"
            set_module_tensor_to_device(self.model, param_name, self.device, value=param, dtype=self.dtype)

    def __call__(self, inputs, output_token):
        # inputs = [(prefix, suffix), ...] with prefix.shape[0] = 1 and suffix.shape[0] = 5
        
        # Reboot the model to make sure buffers are loaded and memory is clean
        del self.model
        clean_memory()
        self.init_model()
        
       # Send batch to device
        batch = [(prefix.to(self.device), suffix.to(self.device)) for prefix, suffix in inputs]
#         n_suffixes = len(batch[0][1])
        n_suffixes = 3
        suffix_eos = [(suffix != self.tokenizer.pad_token_id).sum(1) - 1 for _, suffix in inputs]

        # Create attention mask for the largest input, and position ids to use KV cache
        attention_mask = torch.finfo(self.dtype).min * torch.ones(MAX_LENGTH, MAX_LENGTH)
        attention_mask = attention_mask.triu(diagonal=1)[None, None, ...]
        attention_mask = attention_mask.to(self.device)
        position_ids = torch.arange(MAX_LENGTH, dtype=torch.long, device=self.device)[None, :]

        with torch.inference_mode():

            # Load first layer
            self.load_layer("model.embed_tokens")

            for i, (layer_name, layer) in tqdm(enumerate(zip(self.layer_names, self.layers)), desc=self.device, total=len(self.layers)):

                # Wait for previous layer to be loaded and load next layer
                if (i + 1) < len(self.layer_names):
                    self.load_layer(self.layer_names[i + 1])

                # Run layer
                for j, (prefix, suffix) in enumerate(batch):
                    if layer_name == "model.embed_tokens":
                        batch[j] = (layer(prefix), layer(suffix))
                    elif layer_name == "model.norm":
                        # Only keep the last token at this point
                        batch[j] = (None, layer(suffix[torch.arange(n_suffixes), suffix_eos[j]][:, None]))
                    elif layer_name == "lm_head":
                        batch[j] = layer(suffix)[:, 0, output_token].detach().cpu().numpy()
                    else:
                        # Run prefix
                        len_p, len_s = prefix.shape[1], suffix.shape[1]
                        new_prefix, (k_cache, v_cache) = layer(prefix, use_cache=True, attention_mask=attention_mask[:, :, -len_p:, -len_p:])
                        
                        # Run suffix
                        pos = position_ids[:, len_p:len_p + len_s].repeat(n_suffixes, 1)
                        attn = attention_mask[:, :, -len_s:, -len_p - len_s:].repeat(n_suffixes, 1, 1, 1)
                        kv_cache = (k_cache.repeat(n_suffixes, 1, 1, 1), v_cache.repeat(n_suffixes, 1, 1, 1))
                        new_suffix = layer(suffix, past_key_value=kv_cache, position_ids=pos, attention_mask=attn)[0]
                        batch[j] = (new_prefix, new_suffix)

                # Remove previous layer from memory (including buffers)
                layer.to("meta")
                clean_memory() # proposed by CPMP

        # Get scores
        return batch

In [5]:
# Run model on the 2 GPUs

def get_tokens(row, tokenizer):
        system_prefix = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_prefix}"
        instruction = "Your task is to analyze the sentence and its sentment and answer below. If the sentment on the answer is correct respond yes, if it is not correct respond no."
        input_prefix = f"Question: {row['prompt']}\nProposed answer: "
        prompt_prefix = system_prefix.format(instruction=instruction, input_prefix=input_prefix)
        prefix = tokenizer(prompt_prefix, return_tensors="pt", return_attention_mask=False, truncation=True, max_length=MAX_LENGTH)["input_ids"]
        prompt_suffix = [f"{row[letter]}\n\n### Response:\n" for letter in "ABC"]
        suffix = tokenizer(prompt_suffix, return_tensors="pt", return_attention_mask=False, truncation=True, max_length=MAX_LENGTH, padding=True)["input_ids"][:, 1:]
        return prefix, suffix

def run_model(device, df):
    model = ShardedLlama(checkpoint_path, device=f"cuda:{device}")
    f = partial(get_tokens, tokenizer=model.tokenizer)
    inputs = df.apply(f, axis=1).values
    batches = np.array_split(inputs, N_BATCHES)
    outputs = []
    for i, batch in enumerate(batches):
        # Token #4874 is yes.
        outputs += model(batch, output_token=4874)
    return outputs



In [6]:
sentence = ['I hate any one who can hurt you', 'I hate you' , 'Who hates you is right' , 'none can hate you' , 'my day was okey as i met my friend and this is jsut it' ]
# sentence = ['I hate any one who can hurt you',' I hate you' ]

df = pd.DataFrame(columns =['prompt'])
# row_to_append = pd.DataFrame([{'text': sentence}])
# df = pd.concat([df,row_to_append])
df['prompt'] = sentence
df['A']  = 'Postive'
df['B']  = 'Negative'
df['C']  = 'Neutral'

In [7]:
df

,prompt,A,B,C
0,I hate any one who can hurt you,Postive,Negative,Neutral
1,I hate you,Postive,Negative,Neutral
2,Who hates you is right,Postive,Negative,Neutral
3,none can hate you,Postive,Negative,Neutral
4,my day was okey as i met my friend and this is...,Postive,Negative,Neutral


In [8]:
# Run model
with ThreadPoolExecutor() as executor:
    outputs = list(executor.map(run_model, [0, 1], np.array_split(df, 2)))
    outputs = sum(outputs, [])

# Save results


cuda:1:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:0:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:1:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:0:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:1:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:0:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:0:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:1:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:0:   0%|          | 0/83 [00:00<?, ?it/s]

cuda:1:   0%|          | 0/83 [00:00<?, ?it/s]

In [9]:
n = len(df)
for i, scores in enumerate(outputs):
    top3 = np.argsort(scores)[::-1]
    df.loc[i, "prediction"] = " ".join(["PNC"[j] for j in top3])



In [10]:
df

,prompt,A,B,C,prediction
0,I hate any one who can hurt you,Postive,Negative,Neutral,N C P
1,I hate you,Postive,Negative,Neutral,N P C
2,Who hates you is right,Postive,Negative,Neutral,N C P
3,none can hate you,Postive,Negative,Neutral,P N C
4,my day was okey as i met my friend and this is...,Postive,Negative,Neutral,C P N


In [11]:
outputs

[array([10.375, 11.12 , 10.41 ], dtype=float16),
 array([ 8.375, 14.65 ,  7.906], dtype=float16),
 array([ 9.64, 10.66, 10.28], dtype=float16),
 array([11.86, 11.01, 10.87], dtype=float16),
 array([12.84, 10.13, 12.96], dtype=float16)]